In [2]:
#一、标签未标准化，导致 ElasticNet 报收敛警告

In [ ]:
#问题：原始代码直接将价格作为目标变量，数值跨度较大，ElasticNet 报出以下警告：
#ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations...

In [ ]:
#修改建议：对标签进行对数变换（log1p），以稳定模型优化过程。

In [ ]:
data['价格_log'] = np.log1p(data['价格'])

In [ ]:
#二、Box-Cox 变换在合并数据集上进行，可能发生数据泄露

In [ ]:
#问题：Box-Cox 或 Yeo-Johnson 对全部数据（训练 + 测试）做变换：
full_data['建筑面积_norm'] = boxcox(full_data['建筑面积'])

In [ ]:
#修改建议：只在训练集拟合变换器，然后应用到测试集。

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(method='yeo-johnson')
train_data['建筑面积_norm'] = pt.fit_transform(train_data[['建筑面积']])
test_data['建筑面积_norm'] = pt.transform(test_data[['建筑面积']])

In [ ]:
# 三、聚类模型在训练+测试集上统一聚类，可能发生信息泄露

In [ ]:
# 错误做法
full_data['cluster'] = KMeans(n_clusters=5).fit_predict(full_data[features])

In [ ]:
#修改建议：仅在训练集上聚类，然后将模型用于预测集。
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)
train_data['cluster'] = cluster_model.fit_predict(train_data[cluster_features])
test_data['cluster'] = cluster_model.predict(test_data[cluster_features])

In [ ]:
#四、通过标签训练回归生成新特征：detail_pre 和 MDH_pre

In [ ]:
# “标签回归再变成特征”的方式本质上是标签穿透，属于典型的数据泄露
model.fit(X_train, y_train)
full_data['detail_pre'] = model.predict(full_data[features])

In [ ]:
#建议删除该特征，或仅在训练集上构造，测试集完全隔离。
train_data['detail_pre'] = model.fit(...).predict(...)
test_data['detail_pre'] = model.predict(...)

In [ ]:
#五、ElasticNetCV 参数粗糙，alpha/l1_ratio 太少

In [ ]:
ElasticNetCV(alphas=[0.1, 1], l1_ratio=[0.5, 0.7], ...)
#网格太粗，容易错过较优参数组合。

In [ ]:
#修改建议：细化搜索网格以提升模型表现：

from sklearn.linear_model import ElasticNetCV

model = ElasticNetCV(
    alphas=np.logspace(-3, 2, 30),        # 更密集的 alpha
    l1_ratio=np.linspace(0.1, 0.9, 9),    # 更细的 l1_ratio 粒度
    cv=5,
    max_iter=5000,
    random_state=42
)

In [ ]:
#六、手动相关性筛选 + 忽略变量交互，合理性存疑

In [ ]:
cols = corrDf['价格'][abs(corrDf['价格']) > 0.0001].index
#单变量相关性筛选没有考虑多重共线性或变量组合的交互作用。ElasticNet 本身有变量筛选能力，手动筛选可能反而剔除有价值特征。

In [ ]:
#修改建议：加入 VIF 检查冗余变量或完全依赖 ElasticNet 做特征选择。
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = train_data[features]
vif = pd.DataFrame()
vif["feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [ ]:
#七、代码风格问题：可读性有待改进、函数未封装

In [ ]:
#问题：命名混乱，如 sum_train, fullNew，含义不清。多段重复代码可封装为函数。

In [ ]:
#修改建议：封装常用过程，例如：

def normalize_building_area(train_df, test_df, col='建筑面积'):
    pt = PowerTransformer(method='yeo-johnson')
    train_df[col + '_norm'] = pt.fit_transform(train_df[[col]])
    test_df[col + '_norm'] = pt.transform(test_df[[col]])
    return train_df, test_df